In [1]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [2]:
try:
    __confing.execute_notebook('notebooks/data_extract.ipynb')
except:
    pass

In [3]:
import pandas as pd
import numpy as np
import random
import os

from src import data_procces as dp
from src.feature_selection import fast_fill

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [ ]:
def feature_importance(model, X: pd.DataFrame, y: pd.Series, n: int):
    """
    Obtain the top 'n' feature importance values from a given model.
    
    Args:
        model (object): The trained model used for feature importance calculation.
        X (pd.DataFrame): The input dataframe containing the features.
        y (pd.Series): The target variable series.
        n (int): The number of top feature importances to return.
    
    Returns:
        list: A list of the top 'n' feature names with the highest importances.
    """
    
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(X.columns)
    
    target_name = y.name  # Get the name of the target column from the Series y
    
    if target_name in feature_names_original:
        feature_names_original.remove(target_name)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista



def drop_correlated_features(X, threshold=0.5):
    """Find correlated columns in a DataFrame and drop them 
    
    Arguments:
        df (DataFrame): Data to analize.
        threshold (float): Minimun correlation value considered to decide whether
        two columns are correlated or not.
    
    Rreturns:
        A list with non-correlated columns."""
    
    numeric_features = list(X.select_dtypes(include=['int', 'float']).columns)
    random.shuffle(numeric_features)  # Reorder the list randomly
    numeric_data = X[numeric_features].copy()
    corr = numeric_data.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape, dtype=bool), k=1))
    the_drop = [column for column in upper.columns if any(upper[column] >= threshold)]
    no_correlated_columns = list(numeric_data.drop(the_drop, axis=1).columns)
    
    return no_correlated_columns


def save_list(X, file_name):
    with open(file_name, 'w') as file:
        for element in X:
            file.write(str(element) + '\n')
    print(f"file {file_name}.")

In [5]:
# Define the folder path
folder_path = 'FILES/auto'

# Check if the folders already exist
if os.path.exists(folder_path):
    print("Folders already exist. Skipping folder creation.")
else:
    # Create the folders
    os.makedirs(folder_path)
    
    # Check if the folders are created
    if os.path.exists(folder_path):
        print("Folders created successfully.")
    else:
        print("Failed to create folders.")

Folders already exist. Skipping folder creation.


In [ ]:
df=pd.read_csv(r'data\custom\all_waves.csv')
target='pahosp1y'

In [1]:
# These columns were extracted mainly from the documentation of the health domain.
selected = ['patoilt', 'pacncrsurg', 'padrinkcr', 'pamomage', 'paheight', 'pashophlp', 'pawthh', 'paarms', 'papaina', 'paosleep', 'paodngr', 'padadage', 'pawalkr', 'patoilethlp', 'parxhrtat', 'palgmusaa', 'padoctor1y', 'pameds', 'parxdiabo', 'parxdiab', 'paoangry', 'pahrtatte', 'paclims', 'pamealhlp', 'pahosp1y', 'papainfr', 'pabreast', 'paarthre', 'parafaany', 'parxarthr', 'paweight', 'pahipcomp', 'pagrossaa', 'padrinkbd', 'pamammog', 'parfaany', 'pamobilaa', 'pacncrradn', 'paprmem', 'pashop', 'paeat', 'pauppermoba', 'pacage', 'pastroklmt', 'pacholst', 'pabmi', 'parorgnz', 'paurina2y', 'pagender', 'papapsm', 'paprost', 'paopace', 'pabedhlp', 'parxhibp', 'paoplot', 'pamhip', 'padiabe', 'pafall', 'paglasses', 'pamoney', 'pahibpe', 'paoalchl', 'pasight', 'pawalkhlp', 'pahrtatlmt', 'padrinkb', 'pacncrothr', 'pawheeze', 'pasit', 'pastoop', 'pachair', 'paarthlmt', 'padress', 'paeathlp', 'pastroke', 'papainlv', 'parechrtatt', 'pabathehlp', 'pabreath_m', 'pafatigue', 'pabed', 'pacncrchem', 'pamoneyhlp', 'parxlung_m', 'palunglmt_m', 'pabath', 'palowermoba', 'pamedhlp', 'pahearaid', 'pameals', 'parxstrok', 'pafallinj', 'pahipe_m', 'parifaany', 'parjudg', 'paswell', 'pacncrmeds', 'pasmoken', 'padresshlp', 'pahigov', 'pahearing', 'palift', 'pasmokev', 'paagey', 'pacancre', 'pararcare', 'parfcaren']
print(f'number of columns {len(selected)}')

number of columns 107


In [ ]:
df = df.dropna(subset=[target]) 

df = df[selected].copy()

nulls = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (nulls <= 0.5)]

In [ ]:
y = df[target]
X = df.drop(target,axis=1)
X = fast_fill(X)

In [ ]:
params={'max_depth': 28, 'subsample': 0.6559545947005226, 'colsample_bytree': 0.9075616505206274, 'eta': 0.03568753700048324, 'min_child_weight': 2, 'learning_rate': 0.0869908981133205, 'n_estimators': 571}

In [ ]:
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=42)
X_train, y_train = dp.apply_smote(X_train, y_train)
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

model = XGBClassifier()
model.fit(X_train, y_train)

_X=feature_importance(model, X, y, 85)
_X.append('pahosp1y')
print(_X)

In [ ]:
while True:
    state = random.randint(0, 100)
    folder_path = f"FILES/auto/{state}"

    if os.path.exists(folder_path):
        print(f"The folder '{folder_path}' already exists. Generating another random number.")
    else:
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
        break

for i in range(10):
    while True:
        random_number = random.uniform(0.21, 0.24)
        __X=drop_correlated_features(X, threshold=random_number)
        
        X_train, X_test, y_train, y_test=dp.split_data(X[__X], y, test_size=0.2, random_state=state)
        X_train, y_train = dp.apply_smote(X_train, y_train)
        X_train, X_test = dp.apply_standard_scaler(X_train, X_test)
        
        if len(__X) < 39:
            model = XGBClassifier(**params,n_jobs=-1)
            model.fit(X_train, y_train)
            
            y_pred_proba = model.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_pred_proba)
            
            if auc >= 0.9:
                save_list(__X, f'FILES/auto/{state}/F{len(__X)}_AUC_{round(auc,2)}_Index_{i}.txt')
                break

In [6]:
# List to store the content of TXT files
corpus = []
# Iterate through all folders and TXT files
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            with open(file_path, "r") as f:
                content = f.read()
                corpus.append(content)

In [7]:
# Tokenization of words
tokenized_corpus = [word_tokenize(doc) for doc in corpus]

# Custom stopwords that will not be considered as stopwords
custom_stopwords = []

# Applying TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords, lowercase=False)
tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(doc) for doc in tokenized_corpus])

# Obtaining unique terms (words) in the corpus
feature_names = tfidf_vectorizer.get_feature_names_out()

In [8]:
# Print the unique terms and their respective TF-IDF values
for i, doc in enumerate(tokenized_corpus):
    for j, term in enumerate(feature_names):
        tfidf_value = tfidf_matrix[i, j]

In [9]:
# Creating a list of words ordered by TF-IDF without repeating words
words_tfidf = []
word_set = set()
for i, doc in enumerate(tokenized_corpus):
    for j, term in enumerate(feature_names):
        tfidf_value = tfidf_matrix[i, j]
        if tfidf_value > 0 and term not in word_set:
            words_tfidf.append((term, tfidf_value))
            word_set.add(term)

# Sorting the list by TF-IDF value
words_tfidf.sort(key=lambda x: x[1], reverse=True)

# Saving the sorted list of words to a text file
with open("FILES/tfidf.txt", "w") as f:
    for word, tfidf in words_tfidf:
        f.write(f"{word}: {tfidf}\n")

- The most important features were obtained.
- Correlated columns were dropped.
- Columns that yielded an AUC of 90 or higher were saved.
- TF-IDF was calculated for the columns and sorted based on these values.
- Columns that repeat more frequently have higher predictive power.
- Columns were then selected based on their TF-IDF and whether they could be transformed into questions.